<a href="https://colab.research.google.com/github/AwlMax-11/analyst_task_wakeapp/blob/main/analyst_task_wakeapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
!pip install requests
!pip install pandasql

In [109]:
import requests
import pandas as pd
import io
from pandasql import sqldf

In [110]:
url = 'https://docs.google.com/spreadsheets/d/1jTXLH6jNiPOgm6tArgdGU8oQ86rgHUe-HkVTNp8JiUQ/edit#gid=1343623918'
response = requests.get(url)
pd.set_option('display.max_columns', None)

In [111]:
def read_google_sheet(sheet_id, sheet_gid):
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={sheet_gid}'
    response = requests.get(url)
    data = pd.read_csv(io.BytesIO(response.content))
    return data

In [112]:
sheets = {
    'ReportOffer': ('1jTXLH6jNiPOgm6tArgdGU8oQ86rgHUe-HkVTNp8JiUQ', '1343623918'),
    'Buyer': ('1jTXLH6jNiPOgm6tArgdGU8oQ86rgHUe-HkVTNp8JiUQ', '812659941'),
    'Offer': ('1jTXLH6jNiPOgm6tArgdGU8oQ86rgHUe-HkVTNp8JiUQ', '1126691783'),
    'OfferConversionTarget': ('1jTXLH6jNiPOgm6tArgdGU8oQ86rgHUe-HkVTNp8JiUQ', '1742426209'),
    'ConversionTarget': ('1jTXLH6jNiPOgm6tArgdGU8oQ86rgHUe-HkVTNp8JiUQ', '1024079236')
}

In [113]:
dfs = {}

for sheet_name, (sheet_id, sheet_gid) in sheets.items():
    dfs[sheet_name] = read_google_sheet(sheet_id, sheet_gid)

In [114]:
combined_df = pd.concat(dfs.values(), ignore_index=True)

In [115]:
data = {
    'Sheet': ['ReportOffer', 'Buyer', 'Offer', 'OfferConversionTarget', 'ConversionTarget'],
    'URL': ['url1', 'url2', 'url1', 'url2', 'url1'],
    'Column1': [1, 2, 3, 4, 5],
    'Column2': ['A', 'B', 'C', 'D', 'E']
}
combined_df = pd.DataFrame(data)

In [116]:
dfs_by_sheet = {}

for url, group in combined_df.groupby('URL'):
    group = group.drop(columns=['URL'])
    dfs_by_sheet[url] = group

In [117]:
dfs['ReportOffer']['id'] = dfs['ReportOffer']['id'].astype(int)
dfs['ReportOffer']['keitaro_stream_id'] = dfs['ReportOffer']['keitaro_stream_id'].fillna(0)
dfs['ReportOffer']['keitaro_stream_id'] = dfs['ReportOffer']['keitaro_stream_id'].astype(int)
dfs['ReportOffer']['keitaro_application_id'] = dfs['ReportOffer']['keitaro_application_id'].fillna(0)
dfs['ReportOffer']['keitaro_application_id'] = dfs['ReportOffer']['keitaro_application_id'].astype(int)
dfs['ReportOffer']['cabinet_id'] = dfs['ReportOffer']['cabinet_id'].astype(int)
dfs['ReportOffer']['developer_revenue'] = dfs['ReportOffer']['developer_revenue'].fillna(0)
dfs['ReportOffer']['sub_id'] = dfs['ReportOffer']['sub_id'].fillna(0)

In [118]:
dfs['Offer']['external_id'] = dfs['Offer']['external_id'].fillna(0)
dfs['Offer']['offer_client_id'] = dfs['Offer']['offer_client_id'].fillna(0)
dfs['Offer']['offer_client_id'] = dfs['Offer']['offer_client_id'].astype(int)
dfs['Offer']['responsible_manager_id'] = dfs['Offer']['responsible_manager_id'].fillna(0)
dfs['Offer']['responsible_manager_id'] = dfs['Offer']['responsible_manager_id'].astype(int)
dfs['Offer']['sales_manager_id'] = dfs['Offer']['sales_manager_id'].fillna(0)
dfs['Offer']['sales_manager_id'] = dfs['Offer']['sales_manager_id'].astype(int)
dfs['Offer']['team_id'] = dfs['Offer']['team_id'].fillna(0)
dfs['Offer']['account_manager_team_id'] = dfs['Offer']['account_manager_team_id'].fillna(0)
dfs['Offer']['account_manager_team_id'] = dfs['Offer']['account_manager_team_id'].astype(int)

In [119]:
dfs['OfferConversionTarget']['cohort'] = dfs['OfferConversionTarget']['cohort'].fillna(0)

In [120]:
pysqldf = lambda q: sqldf(q, globals())

In [121]:
report_offer_copy = pd.DataFrame(dfs['ReportOffer'])
offer_copy = pd.DataFrame(dfs['Offer'])
buyer_copy = pd.DataFrame(dfs['Buyer'])
offer_conversion_target_copy = pd.DataFrame(dfs['OfferConversionTarget'])
conversion_target_copy = pd.DataFrame(dfs['ConversionTarget'])

In [122]:
buyer_copy.set_index('id', inplace=True)
offer_copy.set_index('id', inplace=True)
offer_conversion_target_copy.set_index('offer_id', inplace=True)
conversion_target_copy.set_index('id', inplace=True)
report_offer_copy.set_index('date', inplace=True)

In [129]:
sql_query = """
SELECT
    Date,
    Buyer_Name,
    Offer_Title,
    Tracker_Type,
    Offer_Number,
    Country,
    Platform,
    Traffic_Source,
    Total_Revenue,
    Total_Costs,
    Total_Margin,
    COUNT(Total_Conversions) AS Total_Conversions
FROM (
    SELECT
        roc.date AS Date,
        CASE
            WHEN bc.name = 'Default Buyer' THEN 'Ada Lovelace [' || bc.hasoffers_id || ']'
            ELSE bc.name || ' [' || bc.hasoffers_id || ']'
        END AS Buyer_Name,
        oc.title AS Offer_Title,
        oc.tracker AS Tracker_Type,
        roc.offer_id Offer_Number,
        roc.country AS Country,
        roc.platform AS Platform,
        roc.traffic_source AS Traffic_Source,
        roc.payout AS Total_Revenue,
        roc.traffic_cost AS Total_Costs,
        (roc.payout - roc.traffic_cost) AS Total_Margin,
        roc.conversion AS Total_Conversions
    FROM report_offer_copy AS roc
    JOIN buyer_copy AS bc ON roc.buyer_id = bc.id
    JOIN offer_copy AS oc ON roc.offer_id = oc.id
    LEFT JOIN (
        SELECT
            octc.id,
            ctc.id AS conversion_id,
            ctc.code,
            octc.date_to,
            octc.date_since
        FROM offer_conversion_target_copy AS octc
        JOIN conversion_target_copy AS ctc ON octc.conversion_target_id = ctc.id
        WHERE ctc.code IN ('installs', 'is_traffic')
        GROUP BY octc.id, ctc.id, ctc.code
    ) AS conversions ON oc.id = conversions.id
    WHERE
        roc.date >= '2024-04-01'
        AND roc.date < '2024-05-01'
        AND oc.tracker = 'Appsflyer'
        AND roc.corrected_payout IS NOT NULL
        AND (roc.developer_revenue - roc.traffic_cost) != 0.00000
        AND (conversions.date_to IS NULL OR conversions.date_to >= roc.date)
        AND (conversions.date_since IS NULL OR conversions.date_since <= roc.date)
) AS sub
GROUP BY
    sub.Date,
    sub.Buyer_Name,
    sub.Offer_Title,
    sub.Tracker_Type,
    sub.Country,
    sub.Platform,
    sub.Traffic_Source;
"""

result = sqldf(sql_query, locals())
result


,Date,Buyer_Name,Offer_Title,Tracker_Type,Offer_Number,Country,Platform,Traffic_Source,Total_Revenue,Total_Costs,Total_Margin,Total_Conversions
0,2024-04-01,Emperor Palpatine [236],title_208,Appsflyer,134505655921533784,RO,ios,traffic1,0.000000,628.088800,-628.088800,10608
1,2024-04-01,John Doe [263],title_208,Appsflyer,134505655921533784,RO,ios,traffic1,0.000000,6003.790000,-6003.790000,10816
2,2024-04-01,name_455_surname [230],title_208,Appsflyer,134505655921533784,RO,ios,traffic1,0.000000,4017.921000,-4017.921000,2080
3,2024-04-01,name_455_surname [230],title_462,Appsflyer,138394909325266725,UA,ios,traffic1,0.000000,1385.490000,-1385.490000,2540
4,2024-04-01,name_461_surname [238],title_208,Appsflyer,134505655921533784,RO,ios,traffic2,0.000000,3879.372000,-3879.372000,32448
...,...,...,...,...,...,...,...,...,...,...,...,...
248,2024-04-26,name_504_surname [265],title_505,Appsflyer,253995414927914611,BR,android,traffic3,6463.201973,5386.001644,1077.200329,360
249,2024-04-26,name_505_surname [412],title_505,Appsflyer,253995414927914611,CL,android,traffic3,4245.414912,3537.845760,707.569152,2400
250,2024-04-26,name_557_surname [243],title_557,Appsflyer,285976660953456,RU,android,traffic5,5419.547761,4712.650227,706.897534,2704
251,2024-04-26,Ацуши Сакурай [230],title_208,Appsflyer,134505655921533784,RO,ios,traffic1,0.000000,3339.763250,-3339.763250,10816
